# Analyse de Données

## Setup

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import os

sns.set_style('darkgrid')
%matplotlib inline

PROJECT_ROOT_DIR = "."
IMAGES_PATH = os.path.join(PROJECT_ROOT_DIR, "figures")
os.makedirs(IMAGES_PATH, exist_ok=True)

plt.rcParams['figure.figsize'] = (12.8, 9.6)

def save_fig(fig_id, tight_layout=True, fig_extension="png", resolution=300):
    path = os.path.join(IMAGES_PATH, fig_id + "." + fig_extension)
    print("Saving figure", fig_id)
    if tight_layout:
        plt.tight_layout()
    plt.savefig(path, format=fig_extension, dpi=resolution)

In [3]:
#path = '../data'
path = ''
loading = pd.read_csv(path + 'velibLoading.csv', sep=" ")
loading.head()

In [4]:
velibAdds = pd.read_csv(path + 'velibAdds.csv', sep=" ")
velibAdds.head()

In [5]:
velibAdds.describe()

In [6]:
number_of_stations_on_hills = 0
for i in range(1189):
    if velibAdds.iloc[i,2]==1:
        number_of_stations_on_hills+=1
print("{0:.2%}".format(number_of_stations_on_hills/1189))

In [7]:
p = loading.columns.size
Time = np.linspace(1, p, p)

In [8]:
plt.plot(Time, loading.transpose()[1], linewidth=2, color='blue')
plt.xlabel('Time')
plt.ylabel('Loading')
plt.title(velibAdds.names[1])
plt.vlines(x = np.linspace(1, p, 8), ymin=0, ymax=1, colors="black", linestyle="dotted")
save_fig('euryale_station_loading')
plt.show()

In [9]:
plt.figure(figsize=(20, 15))
for i in range(1, 17):
    plt.subplot(4, 4, i)
    plt.plot(Time, loading.transpose()[i], linewidth=2, color='blue')
    plt.xlabel('Time')
    plt.ylabel('Loading')
    plt.title(velibAdds.names[i])
    plt.vlines(x=np.linspace(1, p, 8), ymin=0, ymax=1, colors="black", linestyle="dotted")

#plt.suptitle('Loading of the 16 First Stations', fontsize='x-large')
save_fig('16_first_loading_plots')
plt.show()

In [10]:
def plot_boxplot(x, medianprops, title=None):
    plt.figure(figsize=(24, 12))
    plt.boxplot(x, medianprops=medianprops)
    plt.vlines(x=np.linspace(1, p, 8), ymin=0, ymax=1, colors="blue", linestyle="dotted", linewidth=3)
    plt.xticks(np.arange(0, 168, 5), labels=np.arange(0, 168, 5), fontsize=10)
    if title:
        plt.title(title, fontsize=16)

In [11]:
medianprops = dict(linestyle='-.', linewidth=5, color='firebrick')

plot_boxplot(loading, medianprops, title=None)
save_fig('boxplot_of_variables')
plt.show()

In [12]:
h = 12 # time
t = 3 # shift

plt.scatter(loading.iloc[:, t], loading.iloc[:, t+h])
#plt.title("Loading at time {} vs at time {}".format(t+h, t), fontsize=14)
save_fig('loading_shift_correlation')
plt.show()

In [13]:
fig, axs = plt.subplots(3,4)
for i in range(3):
    for j in range(1,5):
        axs[i,j-1].scatter(loading.iloc[:, i*4+j], loading.iloc[:, i*4+j+1], alpha=.7)
        axs[i,j-1].set_title("time {} vs time {}".format(i*4+j+1, i*4+j))

save_fig('mult_loading_shift_correlation')
plt.show()

In [14]:
corr_matrix = loading.corr()

plt.figure(figsize=(20, 9))
plt.subplot(1, 2, 1)
#plt.title('Total Correlations', fontsize=20)
sns.heatmap(corr_matrix, xticklabels=False, yticklabels=False, cmap="viridis")

plt.subplot(1, 2, 2)
#plt.title('Sub-correlation (Monday)', fontsize=20)
sns.heatmap(corr_matrix.iloc[:24, :24], xticklabels=True, yticklabels=True, cmap="viridis")
save_fig('loading_correlation_total_day')
plt.show()

In [15]:
longitude = velibAdds.longitude
latitude = velibAdds.latitude
on_hill = (velibAdds.bonus == 1) # on recupere les valeurs ou bonus=1 i.e. sur une colline

In [16]:
plt.xlim = (2.2222, 2.4795)
plt.ylim = (48.8064, 48.9152)
plt.scatter(longitude[~on_hill], latitude[~on_hill])
plt.scatter(longitude[on_hill], latitude[on_hill], marker="2")
plt.legend(labels=['Not On Hill', 'On Hill'], fontsize='x-large')
plt.xlabel('Longitudes')
plt.ylabel('Latitudes')
#plt.title('Location of Stations in Paris', fontsize=14)
save_fig('location_of_stations')
plt.show()

In [17]:
plot_boxplot(loading[on_hill], medianprops)
save_fig('boxplot_of_hill_stations')
plt.show()

In [18]:
plot_boxplot(loading[~on_hill], medianprops)
save_fig('boxplot_of_not_hill_stations')
plt.show()

## Principal Component Analysis

Given that we have many variables (168 variables), we use PCA to better understand the meaning of these variables and select only a few of them. <br>
Our data does not need to be scaled before performing PCA as the loading is a number between 0 and 1.

In [19]:
from sklearn.decomposition import PCA

n_components = 15
pca = PCA(n_components=n_components)
loading_pca = pca.fit_transform(loading)

We first plot the percentage of variance explained by the first 15 components.

In [20]:
x = np.arange(pca.explained_variance_.size)
fig, ax = plt.subplots(1, 2, figsize=(20, 10))

ax[0].bar(x, pca.explained_variance_ratio_)
ax[0].plot(pca.explained_variance_ratio_, color='black')
ax[0].set_xlabel('Principal Components', fontsize=16)
ax[0].set_ylabel('Explained Variance', fontsize=16)
#ax[0].set_title('Explained Variance Ratios',fontsize=25)

for p in ax[0].patches:
    text = str(np.round(p.get_height(), 2) * 100) + '%'
    ax[0].annotate(text=text,
                xy=(p.get_x()+p.get_width()/2., p.get_height()+0.01),
                fontsize='large',
                ha='center',
                va='center')

ax[1].bar(x+1, np.cumsum(pca.explained_variance_ratio_), width=.7)
ax[1].set_ylabel('Shared Variance', fontsize=16)
ax[1].set_xlabel('Principal Components', fontsize=16)
#ax[1].set_title('Cumulative Sum of Variance Ratio',fontsize=25)

for p in ax[1].patches:
    text = str(np.round(p.get_height(), 2) * 100) + '%'
    ax[1].annotate(text=text,
                xy=(p.get_x()+p.get_width()/2., p.get_height()+0.01),
                fontsize='large',
                ha='center',
                va='center')

save_fig('explained_var_ratio_and_cumulative')
plt.show()

Most of the variance is contained in the first two components (around 60%), but there is still some important imformation in some of the following components. We explain 80% of the variance with the first seven components. <br>
On the graph 'Explained Variance Ratios', the curve of explained variance ratios is bent over the 6th principal component.<br>
We therefore choose to work with the first 6 components, given that after the sixth component, the cumulated sum of shared variance increases very slowly.

We also analyze the boxplot of the coordinates of the bike stations on the first fifteen principal axis.

In [21]:
plt.boxplot(loading_pca)
plt.axhline(0, c='b', alpha=.2)
plt.xlabel('Principal Components', fontsize=16)
#plt.title('Boxplot of Principal Components',fontsize='xx-large')
save_fig('boxplot_of_pca')
plt.show()

These boxplots give us the same information as before. The first two components are the ones with the biggest variance, and after the sixth component, variance decreases very slowly. <br>
The medians are centered on 0 after the 3rd principal component and the extent of boxplots is constant after the 6th principal component. <br>

The following figure shows the projection of all of the bike stations onto the plane formed by the first two principal axis.

In [22]:
plt.scatter(loading_pca[:, 0][~on_hill], loading_pca[:, 1][~on_hill],
            color='b', label='Not On Hill')
plt.scatter(loading_pca[:, 0][on_hill], loading_pca[:, 1][on_hill],
            color='r', label='On Hill')
plt.axhline(0, linestyle='--', color='black')
plt.axvline(0, linestyle='--', color='black')
plt.xlabel('Dimension 1')
plt.ylabel('Dimension 2')
#plt.title('Individuals Map - PCA')
plt.legend()
plt.grid(True)
save_fig('map_of_individuals')
plt.show()

We observe that there might be a cluster near the bottom left corner, and amongst this cluster are most of the bike stations located on a hill.

We then plot the circle of correlation for the first two components, which we zoom in on to better appreciate.

In [23]:
coord1 = pca.components_[0] * np.sqrt(pca.explained_variance_[0])
coord2 = pca.components_[1] * np.sqrt(pca.explained_variance_[1])

for i, j, nom in zip(coord1,coord2, loading.columns):
    plt.text(i, j, nom)
    plt.arrow(0, 0, i, j, color='black', alpha=.2)

#plt.title('Variables Factor Map - PCA', fontsize=14)
plt.xlabel('Dimension 1 ')
plt.ylabel('Dimension 2 ')
plt.grid(True)
save_fig('variables_correlation_1_2', resolution=250)
plt.show()

We observe that all of the variables take positive values on the first principal axis. For the second axis, positive values tend to correspond to daytime hours (14 to 18), while negative values correspond to nighttime hours (23 to 05). However, all of the variables are very far away from the edge of the circle of radius 1. Therefore we can deduce that :

Principal Component 1: overall loading 

Principal Component 2: contrast between day and night

We also plot the circle of correlation for components 1 and 3.

In [24]:
coord3 = pca.components_[2] * np.sqrt(pca.explained_variance_[2])

for i, j, nom in zip(coord1, coord3, loading.columns):
    plt.text(i, j, nom)
    plt.arrow(0, 0, i, j, color='black', alpha=.1)

#plt.title('Variables Factor Map - PCA', fontsize=14)
plt.xlabel('Dimension 1')
plt.ylabel('Dimension 3')
plt.grid(True)
save_fig('variables_correlation_1_3', resolution=250)
plt.show()

By observing the third principal component, we find that most of the variables with positive coordinates correspond to the days of the weekend, while most of the varibles with negative coordinates correspond to weekdays. We can conclude that:

Principal Component 3: contrast between weekdays and the weekend.

However, it is important to remember that this analysis is mostly inconclusive because the arrows are all very short in norms, which means that the correlation with either dimension is not very strong. 

In [25]:
fig, axs = plt.subplots(6, 1)
x = np.arange(len(pca.components_[1]))
for i in range(6):
    axs[i].plot(x, pca.components_[i])
    axs[i].set_ylabel("PC " + str(i+1))
    
#fig.suptitle('Principal Components', fontsize=14)
save_fig('graphs_pca')
plt.show()

In [26]:
label_dic_hill = {1: 'Not On Hill', 2: 'On Hill'}

def plot_pca(l_pca, fig, ax, nbc, nbc2):
    for i in range(2):
        xs = l_pca[velibAdds.bonus == i, nbc - 1]
        ys = l_pca[velibAdds.bonus == i, nbc2 - 1]
        label = label_dic_hill[i + 1]
        color = cmaps(i)
        ax.scatter(xs, ys, color=color, alpha=.8, s=1, label=label)
        ax.set_xlabel("PC %d: %.2f%%" % (nbc, pca.explained_variance_ratio_[nbc - 1] * 100), fontsize=10)
        ax.set_ylabel("PC %d: %.2f%%" % (nbc2, pca.explained_variance_ratio_[nbc2 - 1] * 100), fontsize=10)

In [27]:
cmaps = plt.get_cmap("Set1")

fig = plt.figure()
for nbc, nbc2, count in [(1, 2, 1), (1, 3, 2), (1, 4, 3), (2, 3, 5), (2, 4, 6), (3, 4, 9)] :
    ax = fig.add_subplot(3, 3, count)
    plot_pca(loading_pca, fig, ax, nbc, nbc2)

plt.legend(loc='best', bbox_to_anchor=(1.8, 2), markerscale=8, fontsize='large')
#plt.suptitle('Principal Components from 1 to 4', fontsize=14)
save_fig('scatter_of_1_4_pc')
plt.show()

## CAH sur les données brutes

In [28]:
import scipy.cluster.hierarchy as sch

height = 30.5
Z = sch.linkage(loading, method="ward", metric="euclidean")
plt.figure(figsize=(20, 10))
sch.dendrogram(Z, no_labels=True)
plt.axhline(height, color='black', alpha=.8, linestyle='--')
plt.ylabel('Euclidean Distance', fontsize='x-large')
#plt.title("Ascending Hierarchical Clustering : Dendrogram", fontsize='xx-large')
save_fig('AHC_dendrogram_raw')
plt.show()

In [29]:
heights = Z[:, 2]  # Décroissance des sauts
x = np.arange(len(heights)) + 1
heights = sorted(heights, reverse=True)
plt.plot(x[:16], heights[:16], '-o')
plt.xlabel('Index')
plt.ylabel('Height',fontsize=14)
plt.axvline(5, color='black', alpha=3, linestyle='--')
#plt.title("Number of clusters",fontsize='xx-large')
save_fig('AHC_heights_raw')
plt.show()

In [30]:
groups_cah = sch.fcluster(Z, t=height, criterion='distance')
newvelib = velibAdds.copy()
newvelib['group'] = groups_cah

In [31]:
n_clusters = np.max(groups_cah)
print('Number of clusters:', n_clusters)

In [62]:
import plotly.graph_objects as go
import plotly.offline as pof

colors = sns.color_palette("husl", n_clusters).as_hex()
grouped = newvelib.groupby('group')
mapbox_access_token = "pk.eyJ1IjoibGFsb3VlYSIsImEiOiJja2tpaGdsbW0weWplMnZxdHl5cG1ncW8xIn0.NG1rULNFLX4zplmUV0WPoQ"

fig = go.Figure()
for i in range(n_clusters):
    df_group = grouped.get_group(i+1)
    fig.add_trace(
        go.Scattermapbox(
            name='Cluster {}'.format(i+1),
            lon=df_group.longitude.values,
            lat=df_group.latitude.values,
            mode='markers',
            marker=dict(size=7, color=colors[i],opacity=0.8),
            hovertemplate=df_group.names,
        )
    )
fig.update_layout(
    title_text='CAH Clusters of Velib Stations',
    mapbox=go.layout.Mapbox(
        accesstoken=mapbox_access_token,
        zoom=12,
        center={'lon':2.3425123048878604, 'lat':48.85920718597386}
    ),
    autosize=False,
    width=1500,
    height=850,
)
pof.iplot(fig)

In [33]:
markers = ['v', '^', 'o', 's', 'p', '*']

for i, marker in zip(np.arange(n_clusters), markers):
    temp = grouped.get_group(i+1)
    plt.scatter(temp.longitude.values, temp.latitude.values, color=colors[i], label='Cluster {}'.format(i+1), marker=marker)

plt.xlabel('Longitude')
plt.ylabel('Latitude')
#plt.title('CAH clusters')
plt.legend(fontsize=16)
save_fig('CAH_clusters_raw')
plt.show()

## CAH sur les CP de l'ACP

In [34]:
height = 32
loadingCP = loading_pca[:, :6]
Z = sch.linkage(loadingCP, method="ward", metric="euclidean")
plt.figure(figsize=(20, 10))
sch.dendrogram(Z, no_labels=True)
plt.axhline(height, color='black', alpha=.5, linestyle='--')
plt.ylabel('Distance')
#plt.title("CAH", fontsize='xx-large')
plt.savefig("CAH_PCA")
plt.show()

In [35]:
groups_cah_acp = sch.fcluster(Z, t=height, criterion='distance')
newvelib = velibAdds.copy()
newvelib['group'] = groups_cah_acp
n_clusters = np.max(groups_cah_acp)

In [36]:
colors = sns.color_palette("husl", n_clusters).as_hex()
grouped = newvelib.groupby('group')
mapbox_access_token = "pk.eyJ1IjoibGFsb3VlYSIsImEiOiJja2tpaGdsbW0weWplMnZxdHl5cG1ncW8xIn0.NG1rULNFLX4zplmUV0WPoQ"
dicts = {}

fig = go.Figure()
for i in range(n_clusters):
    df_group = grouped.get_group(i+1)
    fig.add_trace(
        go.Scattermapbox(
            name='Cluster {}'.format(i+1),
            lon=df_group.longitude.values,
            lat=df_group.latitude.values,
            mode='markers',
            marker=dict(size=5, color=colors[i]),
            hovertemplate=df_group.names,
            showlegend=True
        )
    )
fig.update_layout(
    title_text='CAH Clusters of Velib Stations',
    mapbox=go.layout.Mapbox(
        accesstoken=mapbox_access_token,
        zoom=12,
        center={'lon':2.3425123048878604, 'lat':48.85920718597386}
    ),
    autosize=False,
    width=1500,
    height=850,
)
pof.iplot(fig)

In [37]:
def cross_table(classe1, classe2):
    table = pd.crosstab(classe1, classe2, 
                        rownames=['classes ACP'], colnames=['classes données brutes'])
    a = np.zeros(np.shape(table)[0])
    b = np.zeros(np.shape(table)[0])
    for j in range (0, np.shape(table)[0]):
        for i in range (0, np.shape(table)[0]):
            if a[j] < table[i][j]:
                a[j] = table[i][j]
                b[j] = i                       
                                             
    print ("")
    print ("max colonne", a)
    print ("j=", b)
    print ("")
    #tablebis = np.copy(table)
    n=np.shape(table)[0]
    tablebis= pd.DataFrame(columns=[str(i) for i in range(n)])

    for k in range(n):
        tablebis[str(k)] = table[b[k]][:]
        #tablebis[k][:] = table[b[k]][:]  
          
    return tablebis

In [38]:
cross_table(groups_cah-1, groups_cah_acp-1)

En sélectionnant le même nombre de classes : 4, la CAH sur les 6 CP de l'ACP et la CAH sur les données brutes donnent les mêmes résultats ie elles classent les individus dans les mêmes clusters.

## K-Means

In [39]:
from sklearn.cluster import KMeans

We apply the $k$-means method to regroup the individuals projected into the space formed by the first six principal axes.

In [40]:
loadingCP = loading_pca[:, 0:6]
kmeans = KMeans(n_clusters=n_clusters, random_state=0).fit(loadingCP)
kclassesACP = kmeans.labels_

pd.DataFrame(kclassesACP).hist()
save_fig('histplot_of_3_axes')
plt.show()

We do the same, but this time using the individuals in the full space $\mathbb{R}^{168}$.

In [41]:
from sklearn.preprocessing import scale

kmeans = KMeans(n_clusters=n_clusters, random_state=0).fit(scale(loading))
kclasses = kmeans.labels_
pd.DataFrame(kclasses).hist()
save_fig('histplot_of_raw_data_classes')
plt.show()

Using the function cross_table() we compare these two groups of clusters.

In [42]:
cross_table(kclassesACP, kclasses)

The cross table shows that these two groups of clusters are practically the same. This can also be observed by comparing the two histograms.

We plot the indiviuals into the plane formed by the first two principal axis and we separate the previous clusters by color.

In [43]:
pc1 = loading_pca[:, 0]
pc2 = loading_pca[:, 1]
coul = ['b', 'r', 'g', 'y', 'm', 'c']
n_clusters = 6
# Choix des options
kmeans = KMeans(n_clusters=n_clusters, random_state=42)
# Exécution de l'algorithme
y_pred = kmeans.fit_predict(loadingCP)

centers = kmeans.cluster_centers_

for i, j, indcoul in zip(pc1, pc2, kclasses):
    plt.text(i, j, "o", color=coul[indcoul], fontweight='black')

plt.axis((-8, 8, -8, 8))
plt.scatter(centers[:, 0], centers[:, 1], c='black', s=200, alpha=0.5)
#plt.title('Individuals by Class Factor Map - PCA (with plotted cluster centroids)', fontsize=14)
plt.xlabel('Dimension 1')
plt.ylabel('Dimension 2')
save_fig('clusters_with_centers')
plt.show()

In [44]:
kmeans = KMeans(n_clusters=n_clusters, random_state=42)
y_pred = kmeans.fit_predict(scale(loading))
centers = kmeans.cluster_centers_

plt.figure(figsize=(24, 12))
plt.boxplot(scale(loading), medianprops=medianprops)
plt.boxplot(centers, medianprops=dict(linestyle='dashed', linewidth=5, color='crimson'))
plt.xlabel('Time')
plt.ylabel('Loading')
#plt.title('K-Means on Raw Data')
save_fig('kmeans_raw')
plt.show()

In [45]:
p = loading.columns.size
fig, ax = plt.subplots(3, 2, figsize=(20, 8))

arr = [(i, j) for i in [0, 1, 2] for j in [0, 1]]
k = 0
for i, j in arr:
    ax[i, j].plot(Time, centers[k], linewidth=2, color=coul[k])
    ax[i, j].vlines(x=np.linspace(1, p, 8), ymin=np.min(centers[k]), ymax=np.max(centers[k]),
                color="black", linestyle="dotted", linewidth=3)
    ax[i, j].set_xlabel('Time')
    ax[i, j].set_ylabel('Loading ' + str(k))
    ax[i, j].set_title(f'Mean Loading of Cluster {k+1}')
    k += 1

save_fig('loading_of_clusters')
fig.show()

In [46]:
clusters = [[[0]],[[0]],[[0]],[[0]]]
for k in range(len(y_pred)):
    num = y_pred[k]
    clusters[num].append([loading.iloc[k, :]])

In [47]:
fig, ax = plt.subplots(2, 2, figsize=(20, 8))

ax[0, 0].boxplot(clusters[0], medianprops=dict(linestyle='dashed', linewidth=5, color=coul[0]))
ax[0, 0].set_xlabel('Time')
ax[0, 0].set_ylabel('Loading ' + str(0))

ax[0, 1].boxplot(clusters[1], medianprops=dict(linestyle='dashed', linewidth=5, color=coul[1]))
ax[0, 1].set_xlabel('Time')
ax[0, 1].set_ylabel('Loading ' + str(1))

ax[1, 0].boxplot(clusters[2], medianprops=dict(linestyle='dashed', linewidth=5, color=coul[2]))
ax[1, 0].set_xlabel('Time')
ax[1, 0].set_ylabel('Loading ' + str(2))

ax[1, 1].boxplot(clusters[3], medianprops=dict(linestyle='dashed', linewidth=5, color=coul[3]))
ax[1, 1].set_xlabel('Time')
ax[1, 1].set_ylabel('Loading ' + str(3))

fig.show()

In [48]:
import matplotlib
from sklearn.metrics import silhouette_score, silhouette_samples
from matplotlib.ticker import FixedLocator, FixedFormatter

X = loading_pca.copy()
kmeans_per_k = [KMeans(n_clusters=k, random_state=42).fit(X) for k in range(1, 10)]
inertias = [model.inertia_ for model in kmeans_per_k]
silhouette_score(X, kmeans.labels_)
silhouette_scores = [silhouette_score(X, model.labels_) for model in kmeans_per_k[1:]]

In [49]:
plt.figure(figsize=(11, 9))

for k in (3, 4, 5, 6):
    plt.subplot(2, 2, k - 2)
    
    y_pred = kmeans_per_k[k - 1].labels_
    silhouette_coefficients = silhouette_samples(X, y_pred)

    padding = len(X) // 30
    pos = padding
    ticks = []
    for i in range(k):
        coeffs = silhouette_coefficients[y_pred == i]
        coeffs.sort()
        cmap = matplotlib.cm.get_cmap("Spectral")
        color = cmap(i / k)
        plt.fill_betweenx(np.arange(pos, pos + len(coeffs)), 0, coeffs,
                          facecolor=color, edgecolor=color, alpha=0.7)
        ticks.append(pos + len(coeffs) // 2)
        pos += len(coeffs) + padding

    plt.gca().yaxis.set_major_locator(FixedLocator(ticks))
    plt.gca().yaxis.set_major_formatter(FixedFormatter(range(k)))
    if k in (3, 5):
        plt.ylabel("Cluster")
    
    if k in (5, 6):
        plt.gca().set_xticks([-0.1, 0, 0.2, 0.4, 0.6, 0.8, 1])
        plt.xlabel("Silhouette Coefficient")
    else:
        plt.tick_params(labelbottom=False)

    plt.axvline(x=silhouette_scores[k - 2], color="red", linestyle="--", label='S.C mean')
    plt.title("$k={}$".format(k), fontsize=16)

save_fig('silhouette_coefficient')
plt.legend()
plt.show()

In [50]:
plt.figure(figsize=(8, 5))
plt.plot(range(1, 10), inertias, "bo-")
plt.xlabel("$k$", fontsize=14)
plt.ylabel("Inertia", fontsize=14)
#plt.title('Intertia vs Number of classes')
save_fig('inertia_classes')
plt.show()

We have plotted the silhouette coefficients according to the number of clusters. It is clear that using 5 or 6 clusters is a bad idea because some of the clusters have a lower than average silhouette coefficient. The answer is ambiguous, since the plots with 3 or 4 clusters reveal that both models are adequate. Both models do have misplaced samples and the 3 cluster model does have a cluster that contain a lot of samples. It also has a better average silhouette coefficient. The 4 clusters model does have more well distributed samples.

### Gaussian mixture model on PCA data

In [51]:
from sklearn.mixture import GaussianMixture

In [52]:
from matplotlib.colors import LogNorm


def plot_centroids(centroids, weights=None, circle_color='w', cross_color='k'):
    if weights is not None:
        centroids = centroids[weights > weights.max() / 10]
    plt.scatter(centroids[:, 0], centroids[:, 1],
                marker='o', s=30, linewidths=8,
                color=circle_color, zorder=10, alpha=0.9)
    plt.scatter(centroids[:, 0], centroids[:, 1],
                marker='x', s=50, linewidths=50,
                color=cross_color, zorder=11, alpha=1)


def plot_gaussian_mixture(clusterer, X, resolution=1000, show_ylabels=True):
    mins = X.min(axis=0) - 0.1
    maxs = X.max(axis=0) + 0.1
    xx, yy = np.meshgrid(np.linspace(mins[0], maxs[0], resolution),
                         np.linspace(mins[1], maxs[1], resolution))
    Z = -clusterer.score_samples(np.c_[xx.ravel(), yy.ravel()])
    Z = Z.reshape(xx.shape)

    plt.contourf(xx, yy, Z,
                 norm=LogNorm(vmin=1.0, vmax=30.0),
                 levels=np.logspace(0, 2, 12))
    plt.contour(xx, yy, Z,
                norm=LogNorm(vmin=1.0, vmax=30.0),
                levels=np.logspace(0, 2, 12),
                linewidths=1, colors='k')

    Z = clusterer.predict(np.c_[xx.ravel(), yy.ravel()])
    Z = Z.reshape(xx.shape)
    plt.contour(xx, yy, Z,
                linewidths=2, colors='r', linestyles='dashed')
    
    plt.plot(X[:, 0], X[:, 1], 'k.', markersize=2)
    plot_centroids(clusterer.means_, clusterer.weights_)

    plt.xlabel("Dimension 1", fontsize=14)
    if show_ylabels:
        plt.ylabel("Dimension 2", fontsize=14)
    else:
        plt.tick_params(labelleft=False)

In [53]:
gm = GaussianMixture(n_components=n_clusters, n_init=10, random_state=42)
gm.fit(loading_pca[:, :2])
labels_gm = gm.predict(loading_pca[:, :2]) 

plt.figure(figsize=(12, 8))
plot_gaussian_mixture(gm, loading_pca[:, :2])
save_fig("Gaussian_mix_1")
plt.show()

In [54]:
plt.scatter(X[:, 0], X[:, 1], c=labels_gm, s=40, cmap='viridis')
#plt.title('Gaussian mixture clustering - PCA (with plotted cluster centroids)', fontsize=14)
plt.xlabel('Dimension 1')
plt.ylabel('Dimension 2')
save_fig("Gaussian_mix_2")
plt.show()

In [55]:
gms_per_k = [GaussianMixture(n_components=k, n_init=10, random_state=42).fit(loading_pca[:, :2])
             for k in range(1, 11)]

In [56]:
bics = [model.bic(loading_pca[:, :2]) for model in gms_per_k]
aics = [model.aic(loading_pca[:, :2]) for model in gms_per_k]

In [57]:
plt.figure(figsize=(12, 8))
plt.plot(range(1, 11), bics, "bo-", label="BIC")
plt.plot(range(1, 11), aics, "go--", label="AIC")
plt.xlabel("$k$", fontsize=14)
plt.ylabel("Information Criterion", fontsize=12)
plt.axis([1, 9.5, np.min(aics) - 50, np.max(aics) + 50])
plt.legend()
plt.show()

In [58]:
min_bic = np.infty

for k in range(1, 11):
    for covariance_type in ("full", "tied", "spherical", "diag"):
        bic = GaussianMixture(n_components=k, n_init=10,
                              covariance_type=covariance_type,
                              random_state=42).fit(X).bic(X)
        if bic < min_bic:
            min_bic = bic
            best_k = k
            best_covariance_type = covariance_type

In [59]:
best_k

In [60]:
best_covariance_type

## A faire : tables de contingence : comparaison des différentes classifications 

In [61]:
cross_table(kclasses, groups_cah-1)